In [5]:
import boto3
import re
import os
import numpy as np
import pandas as pd
import io
import itertools
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import sagemaker as sage
from time import gmtime, strftime

In [14]:
bucket = 'allianz-vbr'
prefix = 'prophet-model'

In [7]:
role = get_execution_role()

In [17]:
role

'arn:aws:iam::709891711940:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [8]:
sess = sage.Session()

In [13]:
data_location = "s3://allianz-vbr/prophet-model/data"

In [10]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "709891711940.dkr.ecr.us-east-1.amazonaws.com/prophet-model:latest"

In [15]:
model = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess
)

In [24]:
predictor = model.fit(data_location)

2022-10-10 17:23:08 Starting - Starting the training job...
2022-10-10 17:23:37 Starting - Preparing the instances for trainingProfilerReport-1665422588: InProgress
.........
2022-10-10 17:25:06 Downloading - Downloading input data
2022-10-10 17:25:06 Training - Downloading the training image..Starting the training.
train:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)
train:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["payment_date"] = pd.to_datetime(df['payment_date'].astype(str), format='%Y-%m-%d')
17:25:16 - cmdstanpy - INFO 

In [2]:
import json
import boto3
from datetime import datetime
import tarfile

runtime_client = boto3.client("runtime.sagemaker")
endpoint_name = "prophet-model"

body = {"periods": 27,
        "freq": "W",
        "include_history": False}
from io import BytesIO

js_io = BytesIO(str(body).encode("utf-8"))
res = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                    ContentType="application/json",
                                    Body=js_io)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/prophet-model in account 709891711940 for more information.